# Example of using ioh-explainer

First we need to define the configuration space of an algorithm. We take a standard PSO algorithm as example.

In [1]:
from ConfigSpace import ConfigurationSpace
from ConfigSpace.util import generate_grid

cs = ConfigurationSpace({
    "F": (0.05, 2.0),              # Uniform float
    "CR" : (0.05, 2.0),                    # Uniform float
})

steps_dict = {
    "F": 20, 
    "CR" : 20,
}

In [2]:
from ioh_explainer.core import explainer
import pyswarms as ps
from modde import ModularDE, Parameters
import numpy as np


def run_de(func, config, budget, dim, *args, **kwargs):
    item = {'F': np.array([float(config.get('F'))]), 'CR':np.array([float(config.get('CR'))]),  'lambda_' : 10*dim }
    c = ModularDE(func, **item)
    try:
        c.run()
        return []
    except Exception as e:
        print(f"Found target {func.state.current_best.y} target, but exception ({e}), so run failed")
        return []

de_explainer = explainer(run_de, 
                 cs , 
                 optimizer_args = None,
                 dims = [5,10],#, 10, 20, 40 
                 fids = [1,5], #,5
                 iids = 10, #20 
                 reps = 5, 
                 sampling_method = "grid",  #or random
                 grid_steps_dict = steps_dict,
                 sample_size = None,  #only used with random method
                 budget = 10000, #10000
                 seed = 1,
                 verbose = True)
de_explainer.run(paralell=True)

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


<class 'ioh_explainer.core.explainer'>
Evaluating 400 configurations.


  5%|▌         | 20/400 [00:30<12:14,  1.93s/it]

In [ ]:

de_explainer.plot()


[inf inf inf ... inf inf inf]


XGBoostError: [13:42:37] ../src/data/data.cc:461: Check failed: valid: Label contains NaN, infinity or a value too large.
Stack trace:
  [bt] (0) /home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x20b1e3) [0x7fb59f8a61e3]
  [bt] (1) /home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x218125) [0x7fb59f8b3125]
  [bt] (2) /home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x219660) [0x7fb59f8b4660]
  [bt] (3) /home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(XGDMatrixSetInfoFromInterface+0xad) [0x7fb59f7d5eed]
  [bt] (4) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call_unix64+0x4c) [0x7fb722927dae]
  [bt] (5) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call+0x22f) [0x7fb72292771f]
  [bt] (6) /usr/lib/python3.8/lib-dynload/_ctypes.cpython-38-x86_64-linux-gnu.so(_ctypes_callproc+0x7ff) [0x7fb722b3d8ef]
  [bt] (7) /usr/

In [ ]:
import xgboost
import shap
df = de_explainer.df
df = df.rename(columns={"iid": "Instance variance", "seed": "Stochastic variance"})
for fid in de_explainer.fids:
    for dim in de_explainer.dims:
        subdf = df[(df['fid'] == fid) & (df['dim'] == dim)]
        X = subdf[[*de_explainer.config_space.keys(), 'Instance variance', 'Stochastic variance']]
        y = subdf['auc'].values
        print(y)
            
        # train xgboost model on diabetes data:
        bst = xgboost.train({"learning_rate": 0.01}, xgboost.DMatrix(X, label=y), 100)

        # explain the model's prediction using SHAP values on the first 1000 training data samples
        shap_values = shap.TreeExplainer(bst).shap_values(X)
        shap.summary_plot(shap_values, X)